<a href="https://colab.research.google.com/github/bagavijit418/NLP_ALGO_POC/blob/master/Practice_Word_Embeddings_on_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center><u><H1>Word Embeddings on Sentiment Analysis</H1></u></center>

In [0]:
!pip install numpy==1.16.2


     |████████████████████████████████| 17.3MB 3.4MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: numpy 1.16.4
    Uninstalling numpy-1.16.4:
      Successfully uninstalled numpy-1.16.4


<module 'numpy.version' from '/usr/local/lib/python3.6/dist-packages/numpy/version.py'>


In [0]:
import numpy as np
print(np.__version__)

1.16.2


In [0]:
import tensorflow as tf
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, Dropout, Flatten
from keras.datasets import imdb

Using TensorFlow backend.


In [0]:
num_words = 10000

In [0]:
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=num_words)

In [0]:
print(len(X_train), 'train_data')
print(len(X_test), 'test_data')

25000 train_data
25000 test_data


In [0]:
X_train[0:10]

array([list([1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]),
       list([1, 194, 1153, 194, 8255, 78, 228,

In [0]:
y_train[0:10]

array([1, 0, 0, 1, 0, 0, 1, 0, 1, 0])

In [0]:
max_len = 256
embedding_size = 32
batch_size = 128

## Sequences per time

In [0]:
print(X_train[0])

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]


In [0]:
len(X_train[0])

218

In [0]:
pad  =  'post' #'pre'

In [0]:
#Convert our lists to equal length sequences
X_train_pad = pad_sequences(X_train, maxlen=max_len, padding=pad, truncating=pad)
X_test_pad = pad_sequences(X_test, maxlen=max_len, padding=pad, truncating=pad)

In [0]:
X_train_pad.shape

(25000, 256)

In [0]:
X_train_pad[0]

array([   1,   14,   22,   16,   43,  530,  973, 1622, 1385,   65,  458,
       4468,   66, 3941,    4,  173,   36,  256,    5,   25,  100,   43,
        838,  112,   50,  670,    2,    9,   35,  480,  284,    5,  150,
          4,  172,  112,  167,    2,  336,  385,   39,    4,  172, 4536,
       1111,   17,  546,   38,   13,  447,    4,  192,   50,   16,    6,
        147, 2025,   19,   14,   22,    4, 1920, 4613,  469,    4,   22,
         71,   87,   12,   16,   43,  530,   38,   76,   15,   13, 1247,
          4,   22,   17,  515,   17,   12,   16,  626,   18,    2,    5,
         62,  386,   12,    8,  316,    8,  106,    5,    4, 2223, 5244,
         16,  480,   66, 3785,   33,    4,  130,   12,   16,   38,  619,
          5,   25,  124,   51,   36,  135,   48,   25, 1415,   33,    6,
         22,   12,  215,   28,   77,   52,    5,   14,  407,   16,   82,
          2,    8,    4,  107,  117, 5952,   15,  256,    4,    2,    7,
       3766,    5,  723,   36,   71,   43,  530,  4

## Creating the model:

In [0]:
model = Sequential()

In [0]:
num_words

10000

In [0]:
model.add(Embedding(input_dim=num_words,
                    output_dim=embedding_size,
                    input_length=max_len,
                    name='layer_embedding'))

In [0]:
model.add(Flatten())

In [0]:
model.add(Dense(250, activation='relu'))

In [0]:
model.add(Dropout(0.5))

In [0]:
# Final classification with a sigmoid:
model.add(Dense(1, activation='sigmoid'))

In [0]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [0]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
layer_embedding (Embedding)  (None, 256, 32)           320000    
_________________________________________________________________
flatten_2 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 250)               2048250   
_________________________________________________________________
dropout_3 (Dropout)          (None, 250)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 251       
Total params: 2,368,501
Trainable params: 2,368,501
Non-trainable params: 0
_________________________________________________________________


In [0]:
%%time
model.fit(X_train_pad, y_train, epochs=5, validation_data=(X_test_pad, y_test), batch_size=batch_size)

Train on 25000 samples, validate on 25000 samples
Epoch 1/5
25000/25000 [==============================] - 5s 214us/step - loss: 0.4753 - acc: 0.7544 - val_loss: 0.3313 - val_acc: 0.8579
Epoch 2/5
25000/25000 [==============================] - 1s 51us/step - loss: 0.1687 - acc: 0.9386 - val_loss: 0.3639 - val_acc: 0.8494
Epoch 3/5
25000/25000 [==============================] - 1s 49us/step - loss: 0.0353 - acc: 0.9920 - val_loss: 0.4740 - val_acc: 0.8461
Epoch 4/5
25000/25000 [==============================] - 1s 49us/step - loss: 0.0068 - acc: 0.9993 - val_loss: 0.5639 - val_acc: 0.8474
Epoch 5/5
25000/25000 [==============================] - 1s 50us/step - loss: 0.0020 - acc: 1.0000 - val_loss: 0.6030 - val_acc: 0.8485
CPU times: user 8.14 s, sys: 1.86 s, total: 10 s
Wall time: 10.7 s


## Testing the model:

In [0]:
%%time
eval_ = model.evaluate(X_test_pad, y_test)

25000/25000 [==============================] - 1s 42us/step
CPU times: user 1.3 s, sys: 205 ms, total: 1.5 s
Wall time: 1.04 s


In [0]:
print("Loss: {0:.5}".format(eval_[0]))
print("Accuracy: {0:.2%}".format(eval_[1]))

Loss: 0.60299
Accuracy: 84.85%


## Reference:

https://keras.io/layers/embeddings/